In [1]:
import imagej
import pandas as pd
from io import StringIO
from pathlib import Path

In [2]:
class ImageJWrapper:
    """
    A wrapper class for the ImageJ gateway instance, providing additional convenience methods
    for common image processing tasks and interactions with the ImageJ UI.

    Attributes:
        ij (net.imagej.ImageJ): The ImageJ gateway instance.
    """
    
    def __init__(self, ij_instance):
        """
        Initializes the ImageJWrapper with an ImageJ instance.
    
        Args:
            ij_instance (net.imagej.ImageJ): The ImageJ gateway instance to be wrapped.
        """
        self.ij = ij_instance

    def open_czi_serie(self, image_filename, serie_numbers):
        """
        Opens a CZI file in ImageJ and imports specified series using a macro.
    
        Args:
            image_filename (str): The path to the CZI file to be opened.
            serie_numbers (str): A comma-separated string of series numbers to be imported.
        """

        # Run [ImageJ macro](https://py.imagej.net/en/latest/07-Running-Macros-Scripts-and-Plugins.html#using-imagej-macros-ij-py-run-macro)
        macro = """
#@ String imageFile
#@ String serie_n

run("Bio-Formats Importer", 
    "open=[" + imageFile + "] autoscale color_mode=Composite rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT " + addSerieText(serie_n));

function addSerieText(serieString) {
    // Converted array
    separatedArray = split(serieString,", ");
    
    serieArray = separatedArray;
    for(var i=0;i<separatedArray.length;i++){
        serieArray[i]="series_"+separatedArray[i];
    }
    
    return String.join(separatedArray, ' ')   	
}
        
        """
        
        args = {
            'imageFile': image_filename,
            'serie_n': serie_numbers
        }
        
        self.ij.py.run_macro(macro, args)

    def get_all_open_image_titles(self):
        """
        Retrieves the titles of all currently open images in ImageJ.
    
        Returns:
            List[str]: A list of the titles of all open images in ImageJ.
        """
        # Now let's get all open ImageJ images
        WindowManager = self.ij.WindowManager
        image_titles = WindowManager.getImageTitles()
        
        return list(image_titles)

    def get_image_by_title(self, title):
        """
        Selects an ImageJ image by its title.

        Args:
            title (str): The title of the ImageJ image to be selected.

        Returns:
            ImagePlus: The ImagePlus object representing the selected image, or None if not found.
        """
        WindowManager = self.ij.WindowManager        
        return WindowManager.getImage(title)

    def set_current_image_by_title(self, title):
        """
        Set a the current ImageJ image by its title.

        Args:
            title (str): The title of the ImageJ image to be selected.

        Returns:
            None
        """
        
        WindowManager = self.ij.WindowManager    

        imp = self.get_image_by_title(title)

        WindowManager.setTempCurrentImage(imp)

    def close_all(self):
        """
        Close all open images
        """
        self.ij.IJ.run("Close All")

In [ ]:
# Initialize ImageJ from a local installation
ij = imagej.init('/Fiji.app', mode='interactive')

ij.ui().showUI() # if you want to display the GUI immediately

# Verify the version of ImageJ
print(ij.getVersion())

# Get ImageJ Wrapper instance
ij_wrapper = ImageJWrapper(ij)

In [ ]:
# https://py.imagej.net/en/latest/07-Running-Macros-Scripts-and-Plugins.html#using-scripts-ij-py-run-script
# https://forum.image.sc/t/how-to-get-sub-series-names-of-lif-file-using-bio-formats-importer-in-macro/23573

macro = """
#@ String id
#@output output_info

run("Bio-Formats Macro Extensions");
Ext.setId(id);
Ext.getCurrentFile(file);
Ext.getSeriesCount(seriesCount);

// Initialize an empty array to store concatenated results
concatenatedList = newArray();

output_info = "seriesName, seriesNumber, sizeX, sizeY \\n";

print(seriesCount)
for (s=0; s<seriesCount; s++) {
  Ext.setSeries(s);
  Ext.getSizeX(sizeX);
  Ext.getSizeY(sizeY);
  Ext.getSizeZ(sizeZ);
  Ext.getSizeC(sizeC);
  Ext.getSizeT(sizeT);    
  Ext.getSeriesName(seriesName);
  print(seriesName);
  print("Series #" + (s+1) + " - " + seriesName + ": image resolution is " + sizeX + " x " + sizeY);
  print("Focal plane count = " + sizeZ);
  print("Channel count = " + sizeC);

  // Create a new list/array for each iteration
  currentList = newArray((s+1), sizeX, sizeY);

  //output_info = "Series #" + s + ": image resolution is " + sizeX + " x " + sizeY;
  output_info = output_info + seriesName + ", " + (s+1) + ", " + sizeX + ", " + sizeY + " \\n";
  
  // Concatenate the current list to the concatenatedList
  concatenatedList = Array.concat(concatenatedList, currentList);
}

// Print concatenated list
for (k = 0; k < concatenatedList.length; k++) {
print("Element " + k + ": " + concatenatedList[k]);
}

Ext.close();
"""
        
# args = {
#     'id': '2024_07_08__0460.czi',
# }

# result = ij.py.run_macro(macro, args)

In [24]:
def workflow(filename):

    p = Path(filename)
    
    args = {
        'id': filename,
    }
    
    result = ij.py.run_macro(macro, args)
    
    output_table = ij.py.from_java(result.getOutput("output_info"))
    
    f = StringIO(output_table)
    
    df_info = pd.read_csv(f, delimiter=',')
    print(df_info.head())

    df_info.to_csv(p.stem + '_info.csv',index=False)

    print(p.stem + '_info.csv' + ' saved!')

In [ ]:
filename = '2024_07_08__0460.czi'

workflow(filename)

In [ ]:
import os
import glob

# Specify the directory containing the .czi files
directory = './'

# Use glob to list all .czi files in the directory
czi_files = glob.glob(os.path.join(directory, '*.czi'))

# Loop over each file and perform desired operations
for file in czi_files:
    workflow(file)